# <center> IBM Applied Data Science Capstone<center>

### <center>Capstone Project - The Battle of Neighborhoods<center>

## <center>Opening a bakery in New York City, New York<center>

### <center>Author: Alexandros Vellios<center>

## Table of content

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. [Introduction](#Introduction)
    
2. [Download and Explore the Dataset](#Data)

3. [Analyze Each Neighborhood](#onehot)

4. [Cluster Neighborhoods](#cluster)
    
5. [Examine Clusters](#examine)
    
</font>
</div>

## 1. Introduction
<a id='Introduction'></a>

In this notebook we will try to solve the problem of our client, an independent business owner, who came to us to advise them regarding which neighborhoods within the Bronx borough, in NYC are the best locations to open a bakery.

According to our client we should focus on three features to find the best location:

1. Lack of competition (bakeries)

2. Presence of entertainment businesses near the location

In order to find the best location we will segment and cluster the different neighborhood of Bronx and come up with the best possible location for our client to open his business.

<br/>

Before we start, first we will install all necessary libraries

In [1]:
import numpy as np 
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from geopy.geocoders import Nominatim
import requests
import json
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
print('Libraries imported.')

Libraries imported.


## 2. Download and Explore the Dataset
<a id='Data'></a>

In order to segement the neighborhoods of New York and explore them, we will essentially need a dataset that contains the 5 boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood. 

Luckily, this dataset exists for free on the web.  

Link to the dataset: https://geo.nyu.edu/catalog/nyu_2451_34572

In [2]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


Load and explore the data

In [3]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
newyork_data

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

Next we will transform the data into a pandas dataframe

In [4]:
# define a new variable that includes the neighborhood data
neighborhoods_data = newyork_data['features']

# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

# loop through the data and fill the dataframe
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
# examine the resulting dataframe
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


Let's make sure that all 5 boroughs and 306 neigborhoods are included

In [5]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


Since we are interested only in data from Bronx, we will filter our dataset to create one containing olny Bronx neighborhoods

In [8]:
bronx_data = neighborhoods[neighborhoods['Borough'] == 'Bronx'].reset_index(drop=True)
print(bronx_data.shape)
bronx_data.head()

(52, 4)


,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


Let's visualize the neighborhoods of Bronx

In [53]:
# create map of Bronx using latitude and longitude values
map_bronx = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(bronx_data['Latitude'], bronx_data['Longitude'], bronx_data['Borough'], bronx_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_bronx)  
    
map_bronx

Now let's get the data from Foursquare

In [13]:
CLIENT_ID = 'Z0WNWMTV2E0S0KK1VX1554J4V4TX4LGNACH11V5JX4SHQ4T0' 
CLIENT_SECRET = 'CDXTAOHPIEG4DXMZ0ADSK131ETBK3BEH0WGCVZPJ1QGV0HZZ' 
VERSION = '20180605' 
LIMIT = 100

In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [16]:
# now we create a new dataframe called new york venues
bronx_venues = getNearbyVenues(names=bronx_data['Neighborhood'],
                                   latitudes=bronx_data['Latitude'],
                                   longitudes=bronx_data['Longitude']
                                  )

Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Claremont Village
Concourse Village
Mount Eden
Mount Hope
Bronxdale
Allerton
Kingsbridge Heights


Let's check the size of the resulting dataframe

In [17]:
print(bronx_venues.shape)
bronx_venues.head()

(1212, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
2,Wakefield,40.894705,-73.847201,Walgreens,40.896528,-73.844700,Pharmacy
3,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy
4,Wakefield,40.894705,-73.847201,Dunkin',40.890459,-73.849089,Donut Shop


Let's check how many venues were returned for each neighborhood

In [18]:
bronx_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Allerton,32,32,32,32,32,32
Baychester,24,24,24,24,24,24
Bedford Park,34,34,34,34,34,34
Belmont,100,100,100,100,100,100
Bronxdale,13,13,13,13,13,13
Castle Hill,8,8,8,8,8,8
City Island,29,29,29,29,29,29
Claremont Village,15,15,15,15,15,15
Clason Point,10,10,10,10,10,10


Let's find out how many unique categories can be curated from all the returned venues

In [19]:
print('There are {} uniques categories.'.format(len(bronx_venues['Venue Category'].unique())))

There are 169 uniques categories.


Let's see the unique categories alphabetically sorted

In [22]:
uniqueValues = bronx_venues['Venue Category'].unique()
print(uniqueValues)

['Dessert Shop' 'Ice Cream Shop' 'Pharmacy' 'Donut Shop' 'Gas Station'
 'Sandwich Place' 'Pizza Place' 'Laundromat' 'Discount Store'
 'Mattress Store' 'Bagel Shop' 'Grocery Store' 'Fast Food Restaurant'
 'Restaurant' 'Bus Station' 'Baseball Field' 'Chinese Restaurant'
 'Liquor Store' 'Gift Shop' 'Park' 'Accessories Store'
 'Caribbean Restaurant' 'Diner' 'Seafood Restaurant' 'Deli / Bodega'
 'Bowling Alley' 'Bus Stop' 'Platform' 'Metro Station' 'Convenience Store'
 'Plaza' 'River' 'Bank' 'Food Truck' 'Home Service' 'Gym'
 'Medical Supply Store' 'Playground' 'Gourmet Shop'
 'Latin American Restaurant' 'Burger Joint' 'Pub' 'Beer Bar'
 'Warehouse Store' 'Spanish Restaurant' 'Coffee Shop' 'Wings Joint'
 'Mexican Restaurant' 'Bar' 'Bakery' 'Trail' 'Supermarket' 'Candy Store'
 'Rental Car Location' 'Thrift / Vintage Store' 'Breakfast Spot' 'Café'
 'Fried Chicken Joint' 'Pet Store' 'Supplement Shop' 'Shipping Store'
 'Sports Bar' 'Furniture / Home Store' 'Asian Restaurant'
 'Martial Arts Dojo'

## 3. Analyze Each Neighborhood
<a id='onehot'></a>

In [23]:
# one hot encoding
bronx_onehot = pd.get_dummies(bronx_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
bronx_onehot['Neighborhood'] = bronx_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [bronx_onehot.columns[-1]] + list(bronx_onehot.columns[:-1])
bronx_onehot = bronx_onehot[fixed_columns]

bronx_onehot.head()

,Neighborhood,Accessories Store,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Board Shop,Boat or Ferry,Bowling Alley,Breakfast Spot,Brewery,Buffet,Building,Burger Joint,Bus Line,Bus Station,Bus Stop,Café,Camera Store,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Donut Shop,Eastern European Restaurant,Electronics Store,Event Space,Eye Doctor,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Food,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hotel,Ice Cream Shop,Indian Restaurant,Indie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Liquor Store,Lounge,Market,Martial Arts Dojo,Mattress Store,Medical Supply Store,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Music Venue,Nightclub,Office,Other Great Outdoors,Outdoor Sculpture,Outlet Store,Paella Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Platform,Playground,Plaza,Pool,Pub,Recording Studio,Recreation Center,Rental Car Location,Restaurant,River,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Smoke Shop,Social Club,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tapas Restaurant,Tattoo Parlor,Tennis Court,Tennis Stadium,Thai Restaurant,Thrift / Vintage Store,Trail,Train Station,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Waste Facility,Weight Loss Center,Wine Shop,Wings Joint,Women's Store
0,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [25]:
bronx_onehot.shape

(1212, 170)

Next step is to group rows by neighborhoods and by taking the mean of the frequenct of occurance of each category

In [27]:
bronx_grouped = bronx_onehot.groupby('Neighborhood').mean().reset_index()
bronx_grouped

,Neighborhood,Accessories Store,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Board Shop,Boat or Ferry,Bowling Alley,Breakfast Spot,Brewery,Buffet,Building,Burger Joint,Bus Line,Bus Station,Bus Stop,Café,Camera Store,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Donut Shop,Eastern European Restaurant,Electronics Store,Event Space,Eye Doctor,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Food,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hotel,Ice Cream Shop,Indian Restaurant,Indie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Liquor Store,Lounge,Market,Martial Arts Dojo,Mattress Store,Medical Supply Store,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Music Venue,Nightclub,Office,Other Great Outdoors,Outdoor Sculpture,Outlet Store,Paella Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Platform,Playground,Plaza,Pool,Pub,Recording Studio,Recreation Center,Rental Car Location,Restaurant,River,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Smoke Shop,Social Club,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tapas Restaurant,Tattoo Parlor,Tennis Court,Tennis Stadium,Thai Restaurant,Thrift / Vintage Store,Trail,Train Station,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Waste Facility,Weight Loss Center,Wine Shop,Wings Joint,Women's Store
0,Allerton,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.031250,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.031250,0.000000,0.00,0.000000,0.000000,0.000000,0.031250,0.000000,0.000000,0.00000,0.000000,0.000000,0.031250,0.00,0.062500,0.000000,0.000000,0.000000,0.000000,0.031250,0.000000,0.000000,0.031250,0.062500,0.031250,0.000000,0.031250,0.000000,0.00000,0.00000,0.031250,0.000000,0.000000,0.000000,0.000000,0.000000,0.031250,0.000000,0.00,0.031250,0.000000,0.000000,0.000000,0.031250,0.000000,0.000000,0.031250,0.000000,0.000000,0.000000,0.031250,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.031250,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.031250,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.031250,0.00,0.156250,0.000000,0.031250,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.00,0.0,0.062500,0.031250,0.000000,0.000000,0.000000,0.062500,0.000000,0.000000,0.00,0.000000,0.00,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
1,Baychester,0.041667,0.000000,0.041667,0.00,0.041667,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.000000,0.000000,0.0

In [29]:
bronx_grouped.shape

(52, 170)

Now we will print each neighborhood and their top 10 most common venues 

In [30]:
num_top_venues = 10

for hood in bronx_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = bronx_grouped[bronx_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Allerton----
                   venue  freq
0            Pizza Place  0.16
1                    Spa  0.06
2     Chinese Restaurant  0.06
3       Department Store  0.06
4            Supermarket  0.06
5           Intersection  0.03
6           Dessert Shop  0.03
7     Spanish Restaurant  0.03
8  Check Cashing Service  0.03
9         Cosmetics Shop  0.03


----Baychester----
                  venue  freq
0            Donut Shop  0.08
1    Mexican Restaurant  0.04
2     Electronics Store  0.04
3        Discount Store  0.04
4  Fast Food Restaurant  0.04
5        Sandwich Place  0.04
6           Pizza Place  0.04
7     Convenience Store  0.04
8             Pet Store  0.04
9   Fried Chicken Joint  0.04


----Bedford Park----
                venue  freq
0               Diner  0.12
1  Chinese Restaurant  0.12
2         Bus Station  0.09
3         Pizza Place  0.06
4       Deli / Bodega  0.06
5      Sandwich Place  0.06
6  Spanish Restaurant  0.06
7  Mexican Restaurant  0.06
8         Superm

                venue  freq
0  Frozen Yogurt Shop  0.08
1         Pizza Place  0.08
2  Italian Restaurant  0.08
3  Chinese Restaurant  0.08
4               Plaza  0.04
5        Home Service  0.04
6                 Spa  0.04
7         Bus Station  0.04
8          Donut Shop  0.04
9          Smoke Shop  0.04


----Port Morris----
                           venue  freq
0      Latin American Restaurant  0.13
1         Furniture / Home Store  0.13
2               Storage Facility  0.07
3                    Music Venue  0.07
4                     Donut Shop  0.07
5                     Restaurant  0.07
6                   Cupcake Shop  0.07
7            Peruvian Restaurant  0.07
8  Paper / Office Supplies Store  0.07
9                     Food Truck  0.07


----Riverdale----
                  venue  freq
0           Bus Station  0.17
1                  Park  0.17
2          Home Service  0.08
3            Food Truck  0.08
4                   Gym  0.08
5  Medical Supply Store  0.08
6        Ba

#### Now let's put that info into a *pandas* dataframe

In [31]:
# Function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [32]:
# Now let's create the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = bronx_grouped['Neighborhood']

for ind in np.arange(bronx_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(bronx_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allerton,Pizza Place,Spa,Supermarket,Chinese Restaurant,Department Store,Intersection,Cosmetics Shop,Fast Food Restaurant,Bus Station,Check Cashing Service
1,Baychester,Donut Shop,Accessories Store,Mexican Restaurant,Sporting Goods Shop,Spanish Restaurant,Mattress Store,Bus Station,Men's Store,Fast Food Restaurant,Sandwich Place
2,Bedford Park,Chinese Restaurant,Diner,Bus Station,Mexican Restaurant,Spanish Restaurant,Sandwich Place,Deli / Bodega,Pizza Place,Fried Chicken Joint,Baseball Field
3,Belmont,Italian Restaurant,Pizza Place,Deli / Bodega,Bakery,Dessert Shop,Grocery Store,Donut Shop,Liquor Store,Spanish Restaurant,Bank
4,Bronxdale,Bank,Park,Eastern European Restaurant,Mexican Restaurant,Supermarket,Chinese Restaurant,Gym,Paper / Office Supplies Store,Performing Arts Venue,Italian Restaurant


## 4. Cluster Neighborhoods
<a id='cluster'></a>

In [33]:
# set number of clusters
kclusters = 5

bronx_grouped_clustering = bronx_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bronx_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 1, 3, 0, 0, 1, 1, 0, 1], dtype=int32)

#Next, we create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [35]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

bronx_merged = bronx_data

# merge bronx_grouped with bronx_data to add latitude/longitude for each neighborhood
bronx_merged = bronx_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

bronx_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bronx,Wakefield,40.894705,-73.847201,0,Pharmacy,Ice Cream Shop,Dessert Shop,Pizza Place,Sandwich Place,Laundromat,Gas Station,Donut Shop,Electronics Store,Fish & Chips Shop
1,Bronx,Co-op City,40.874294,-73.829939,1,Bus Station,Baseball Field,Accessories Store,Grocery Store,Gift Shop,Liquor Store,Mattress Store,Fast Food Restaurant,Park,Discount Store
2,Bronx,Eastchester,40.887556,-73.827806,1,Caribbean Restaurant,Diner,Deli / Bodega,Pizza Place,Seafood Restaurant,Bus Station,Bus Stop,Metro Station,Bowling Alley,Fast Food Restaurant
3,Bronx,Fieldston,40.895437,-73.905643,1,Plaza,River,Bus Station,Electronics Store,Food,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Eye Doctor
4,Bronx,Riverdale,40.890834,-73.912585,1,Bus Station,Park,Gym,Food Truck,Playground,Medical Supply Store,Baseball Field,Plaza,Bank,Home Service


Finally, let's visualize our results

In [38]:
address = 'Bronx, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bronx, New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bronx, New York City are 40.8466508, -73.8785937.


In [41]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bronx_merged['Latitude'], bronx_merged['Longitude'], bronx_merged['Neighborhood'], bronx_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters
<a id='examine'></a>

Now, we will examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, we will decide which cluster is the best fit for the bakery based on our clients needs. Finally we will look at the neighborhoods within this cluster to see which one is the best.

#### Cluster 1

In [44]:
bronx_merged.loc[bronx_merged['Cluster Labels'] == 0, bronx_merged.columns[[1] + list(range(5, bronx_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Wakefield,Pharmacy,Ice Cream Shop,Dessert Shop,Pizza Place,Sandwich Place,Laundromat,Gas Station,Donut Shop,Electronics Store,Fish & Chips Shop
5,Kingsbridge,Pizza Place,Bar,Bakery,Latin American Restaurant,Mexican Restaurant,Supermarket,Sandwich Place,Pharmacy,Spanish Restaurant,Donut Shop
7,Norwood,Pizza Place,Park,Bank,Pharmacy,Mexican Restaurant,Mobile Phone Shop,Sandwich Place,Restaurant,Chinese Restaurant,Spanish Restaurant
9,Baychester,Donut Shop,Accessories Store,Mexican Restaurant,Sporting Goods Shop,Spanish Restaurant,Mattress Store,Bus Station,Men's Store,Fast Food Restaurant,Sandwich Place
13,University Heights,Pizza Place,Bakery,Shoe Store,Fried Chicken Joint,Sandwich Place,Supermarket,Chinese Restaurant,Donut Shop,Grocery Store,Bank
14,Morris Heights,Spanish Restaurant,Health & Beauty Service,Bus Station,Bank,Latin American Restaurant,Pizza Place,Grocery Store,Pharmacy,Recreation Center,Event Space
15,Fordham,Spanish Restaurant,Mobile Phone Shop,Fast Food Restaurant,Bank,Shoe Store,Donut Shop,Pizza Place,Pharmacy,Gym / Fitness Center,Sporting Goods Shop
16,East Tremont,Pizza Place,Lounge,Deli / Bodega,Discount Store,Cosmetics Shop,Paella Restaurant,Donut Shop,Shoe Store,Café,Mobile Phone Shop
18,High Bridge,Pharmacy,Pizza Place,Chinese Restaurant,Bus Station,Supermarket,Deli / Bodega,Park,Gym,Asian Restaurant,Seafood Restaurant
19,Melrose,Pizza Place,Pharmacy,Sandwich Place,Discount Store,Gym / Fitness Center,Grocery Store,Paper / Office Supplies Store,Supermarket,Supplement Shop,Donut Shop


This cluster contains a lot of competition (donut shops, cafes etc.) therefore it is excluded from further analysis

#### Cluster 2

In [45]:
bronx_merged.loc[bronx_merged['Cluster Labels'] == 1, bronx_merged.columns[[1] + list(range(5, bronx_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Co-op City,Bus Station,Baseball Field,Accessories Store,Grocery Store,Gift Shop,Liquor Store,Mattress Store,Fast Food Restaurant,Park,Discount Store
2,Eastchester,Caribbean Restaurant,Diner,Deli / Bodega,Pizza Place,Seafood Restaurant,Bus Station,Bus Stop,Metro Station,Bowling Alley,Fast Food Restaurant
3,Fieldston,Plaza,River,Bus Station,Electronics Store,Food,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Eye Doctor
4,Riverdale,Bus Station,Park,Gym,Food Truck,Playground,Medical Supply Store,Baseball Field,Plaza,Bank,Home Service
8,Williamsbridge,Caribbean Restaurant,Nightclub,Soup Place,Bar,Food & Drink Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Eye Doctor
10,Pelham Parkway,Italian Restaurant,Frozen Yogurt Shop,Chinese Restaurant,Pizza Place,Plaza,Performing Arts Venue,Smoke Shop,Bus Station,Sandwich Place,Coffee Shop
11,City Island,Harbor / Marina,Seafood Restaurant,Thrift / Vintage Store,Grocery Store,History Museum,Bar,Diner,Pizza Place,Music Venue,Sandwich Place
12,Bedford Park,Chinese Restaurant,Diner,Bus Station,Mexican Restaurant,Spanish Restaurant,Sandwich Place,Deli / Bodega,Pizza Place,Fried Chicken Joint,Baseball Field
17,West Farms,Bus Station,Bus Stop,Intersection,Basketball Court,Playground,Ice Cream Shop,Cosmetics Shop,Park,Coffee Shop,Donut Shop
21,Port Morris,Furniture / Home Store,Latin American Restaurant,Cupcake Shop,Peruvian Restaurant,Distillery,Restaurant,Paper / Office Supplies Store,Donut Shop,Music Venue,Spanish Restaurant


This cluster contains a lot of competition (donut shops, cafes etc.) therefore it is excluded from further analysis

#### Cluster 3

In [46]:
bronx_merged.loc[bronx_merged['Cluster Labels'] == 2, bronx_merged.columns[[1] + list(range(5, bronx_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,Country Club,Sandwich Place,Playground,Weight Loss Center,Donut Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Eye Doctor,Event Space


For this cluster out of the top 10 venues, 2 are competitors (sandwich and donut shop), while 3 out 10 venues are entertainment businesses

#### Cluster 4

In [47]:
bronx_merged.loc[bronx_merged['Cluster Labels'] == 3, bronx_merged.columns[[1] + list(range(5, bronx_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Woodlawn,Deli / Bodega,Pizza Place,Playground,Pub,Bar,Grocery Store,Park,Rental Car Location,Donut Shop,Food & Drink Shop
27,Throgs Neck,Deli / Bodega,Pizza Place,American Restaurant,Italian Restaurant,Juice Bar,Sports Bar,Asian Restaurant,Coffee Shop,Bar,Event Space
31,Van Nest,Pizza Place,Deli / Bodega,BBQ Joint,Coffee Shop,Donut Shop,Middle Eastern Restaurant,Spa,Supermarket,Board Shop,Bakery
33,Belmont,Italian Restaurant,Pizza Place,Deli / Bodega,Bakery,Dessert Shop,Grocery Store,Donut Shop,Liquor Store,Spanish Restaurant,Bank
38,Edgewater Park,Deli / Bodega,Italian Restaurant,Coffee Shop,Pizza Place,Donut Shop,Bar,Park,Sports Bar,Asian Restaurant,Pub


For this cluster out of the top 10 venues, on average 2.3 are competitors (sandwich shop, cafe, donut shop etc), while 2.5 out 10 venues are entertainment businesses

#### Cluster 5

In [48]:
bronx_merged.loc[bronx_merged['Cluster Labels'] == 4, bronx_merged.columns[[1] + list(range(5, bronx_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,Edenwald,Supermarket,Fish Market,Gas Station,Grocery Store,Women's Store,Event Space,Food,Fish & Chips Shop,Fast Food Restaurant,Farmers Market


The final cluster contains no competitors on the top 10 venues, while 1 out of 10 venues are entertainment businesses

Therefore given the results of the above analysis, we will propose to our client that the best neighborhood in bronx to open his bakery is **Edenwald**